In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [4]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [5]:
EPOCHS = 50
BATCH_SIZE = 64

In [11]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
                   ]),
                  ),
    batch_size=BATCH_SIZE, shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data',
                   train=False,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
                   ]),
                  ),
    batch_size=BATCH_SIZE, shuffle=True,
)

In [13]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p
    
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training,
                     p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training,
                     p=self.dropout_p)
        x = self.fc3(x)
        return x

In [14]:
model = Net(dropout_p = 0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [15]:
def train(model, train_loader, optimizer):
    model.train()
    for b_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    for b_idx, (data, target) in enumerate(test_loader):
        test_loss = 0
        correct = 0
        with torch.no_grad():
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        test_acc = 100. * correct / len(test_loader.dataset)
        return test_loss, test_acc


In [16]:
for epoch in range(EPOCHS):
    train(model, train_loader, optimizer)
    test_loss, test_acc = evaluate(model, test_loader)
    
    print(f"[{epoch} Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}]")

[0 Test Loss: 0.0059, Accuracy: 0.4100]
[1 Test Loss: 0.0051, Accuracy: 0.4300]
[2 Test Loss: 0.0034, Accuracy: 0.5300]
[3 Test Loss: 0.0036, Accuracy: 0.5100]
[4 Test Loss: 0.0035, Accuracy: 0.5300]
[5 Test Loss: 0.0025, Accuracy: 0.5400]
[6 Test Loss: 0.0039, Accuracy: 0.5000]
[7 Test Loss: 0.0027, Accuracy: 0.5400]
[8 Test Loss: 0.0025, Accuracy: 0.5400]
[9 Test Loss: 0.0021, Accuracy: 0.5600]
[10 Test Loss: 0.0035, Accuracy: 0.5200]
[11 Test Loss: 0.0030, Accuracy: 0.4900]
[12 Test Loss: 0.0026, Accuracy: 0.5400]
[13 Test Loss: 0.0025, Accuracy: 0.5100]
[14 Test Loss: 0.0019, Accuracy: 0.5700]
[15 Test Loss: 0.0025, Accuracy: 0.5400]
[16 Test Loss: 0.0024, Accuracy: 0.5700]
[17 Test Loss: 0.0021, Accuracy: 0.5600]
[18 Test Loss: 0.0036, Accuracy: 0.5200]
[19 Test Loss: 0.0029, Accuracy: 0.5500]
[20 Test Loss: 0.0032, Accuracy: 0.5200]
[21 Test Loss: 0.0031, Accuracy: 0.5500]
[22 Test Loss: 0.0020, Accuracy: 0.5600]
[23 Test Loss: 0.0036, Accuracy: 0.4900]
[24 Test Loss: 0.0023, Acc